# Automated Landsat 8 image bulk download and preprocessing

_Last modified 2022-01-07._

This script is run to download and preprocess the Landsat 8 images over the glaciers prior to the Wavelet Transform Modulus Maxima (WTMM) segmentation analysis where the calving front delineations are produced.

The code is streamlined to analyze images for hundreds of glaciers, specifically, the marine-terminating glaciers along the periphery of Greenland. For use on other glaciers, sections of code must be modified:

##########################################################################################

__Indicates code that must be modified__

##########################################################################################

Recommendation is to keep a record of the csv file names generated throughout the preprocessing as many of them will be used later for analysis.

Terminal command line requirements:

 - Contains code to bulk download Landsat 8 images and metadata stored on the Amazon Web Services cloud (s3 bucket) over a region of interest using the Amazon Web Services command line interface. Follow instructions at https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-install.html to get required __aws__ commands onto your command terminal.
 
 - Uses GDAL command line functions. Make sure __gdal__ is installed properly.
 - Uses image magick command line functions. Download instructions available at https://imagemagick.org/script/download.php.


Outline of steps:
1. Set-up: import packages, set paths, and enter glaciers IDs
2. Find all the Landsat footprints that overlap the glaciers
3. Download Landsat metadata (*MTL.txt) files from AWS for all overlapping scenes
4. Calculate cloud % over terminus box using Landsat quality band
5. Create buffer zone around terminus boxes and rasterize terminus boxes
6. Download non-cloudy Landsat images from AWS
7. Calculate weighted average glacier flow direction using velocity data
8. Rotate all images by flow direction
9. Crop all images to the same size

# 1) Set-up: import packages, set paths, and enter glaciers IDs

In [1]:
import numpy as np
import pandas as pd
import scipy
import math
import subprocess
import os
import shutil
import datetime
import cv2
from PIL import Image
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

# geospatial packages
import fiona
import geopandas as gpd
from shapely.geometry import Polygon, Point, LineString
import shapely
import rasterio

# Enable fiona KML file reading driver
gpd.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw'


######################################################################################
# SET YOUR PATHS HERE:
basepath='/home/jukes/Documents/Sample_glaciers/' # folder containing the terminus box shapefile(s)
downloadpath ='/media/jukes/jukes1/LS8aws/' # folder to contain downloaded Landsat 8 images
os.chdir('/home/jukes/automated-glacier-terminus/') # path to this repo
######################################################################################

# import necessary functions from automated-glacier-terminus.py
from automated_terminus_functions import distance, resize_pngs

In [12]:
# !pip install scipy

### Enter in the glacier BoxIDs:

The Greenland peripheral glacier terminus boxes were referenced using their 3 digit BoxID: Box###.
For other glaciers, replace this code with a list of IDs corresponding to the glaciers and corresponding shapefiles (e.g. BoxHelheim.shp). 

In [2]:
######################################################################################

# BoxIDs = []
# boxes = list(map(str, np.arange(1, 642, 1)))
# for BoxID in boxes:
#     BoxID = BoxID.zfill(3)
#     BoxIDs.append(BoxID)
# print(BoxIDs)

# BoxIDs = ['Kangerlussuaq','Bernstorff1','Bernstorff2','Bernstorff3']
BoxIDs = ['HM','Petermann']
print(BoxIDs)
######################################################################################

['HM', 'Petermann']


### Create new folders corresponding to these glaciers:

In [27]:
# create new BoxID folders 
for BoxID in BoxIDs:
    # create folder in Sample_glaciers
    if os.path.exists(basepath+'Box'+BoxID):
#         shutil.rmtree(basepath+'Box'+BoxID) # remove the old folder
        print("Path exists already for Box", BoxID)
    else:
        os.mkdir(basepath+'Box'+BoxID)
            
    # make a new folder for the glacier in the download folder
    if os.path.exists(downloadpath+'Box'+BoxID):
        print("Path exists already in LS8aws for Box", BoxID)
    else:
        os.mkdir(downloadpath+'Box'+BoxID)

#     ######################################################################################
#     # ONLY APPLIES TO GREENLAND PERIPHERAL GLACIERS, MUST MODIFY FOR OTHER GLACIERS
#     # OR COMMENT OUT AND DO MANUALLY
#     boxespath = '/media/jukes/jukes1/Boxes_individual/' # path to terminus box shapefiles
#     RGIpath = '/media/jukes/jukes1/RGI_shps/' # path to RGI glacier outlines
    
#     # move terminus box shapefile into the new folder
#     ID = int(BoxID) # make into an integer in order to grab the .shp files from Boxes_individual   
#     for file in os.listdir(boxespath):
#         if file.startswith(str(ID)):
#             if len(file) == len(str(ID))+4:
# #             if file.endswith('.dbf') or file.endswith('.prj') or file.endswith('.qpj') or file.endswith('.shx') or file.endswith('.shp'):
#                 shutil.copyfile(boxespath+file, basepath+'Box'+BoxID+'/Box'+BoxID+file[-4:])

#     # move RGI glacier outline into the new folder
#     for file in os.listdir(RGIpath):
#         if file.startswith('BoxID_'+str(ID)):
#             shutil.copyfile(RGIpath+file, basepath+'Box'+BoxID+'/RGI_Box'+BoxID+file[-4:])
            
#     ######################################################################################

Path exists already for Box HM
Path exists already in LS8aws for Box HM
Path exists already for Box Petermann
Path exists already in LS8aws for Box Petermann


# 2) Find all the Landsat footprints that overlap the glaciers

This step requires the WRS-2_bound_world_0.kml file containing the footprints of all the Landsat scene boundaries available through the USGS (https://www.usgs.gov/land-resources/nli/landsat/landsat-shapefiles-and-kml-files). Place this file in your base directory (basepath). 

To check if they overlap the glacier terminus box shapefiles, the box shapefiles must be in WGS84 coordinates (ESPG: 4326). 
 - If they are already in WGS84, skip this step and rename them to end in "_WGS.shp"
 - If they are not, we can use the following GDAL command to reproject:

        ogr2ogr -f "ESRI Shapefile" -t_srs EPSG:NEW_EPSG_NUMBER -s_srs EPSG:OLD_EPSG_NUMBER output.shp input.shp

In [42]:
# Reproject terminus box shapefiles to WGS84 if in a different projection
######################################################################################
source_srs = 'EPSG:3413' # current projection of the terminus box shapfiles (Greenland polar stereo = 3413)
######################################################################################

for BoxID in BoxIDs:
    boxpath = basepath+"Box"+BoxID+"/Box"+BoxID # access the BoxID folders created 
    # construct the gdal command
    rp = "ogr2ogr -f 'ESRI Shapefile' -t_srs EPSG:4326 -s_srs "+source_srs+" "+boxpath+"_WGS.shp "+boxpath+".shp"
    print(rp)
    subprocess.call(rp, shell=True) # run the command on terminal
#     print("Box"+BoxID+" done.") # keep track of progress

ogr2ogr -f 'ESRI Shapefile' -t_srs EPSG:4326 -s_srs EPSG:3413 /Users/jukesliu/Documents/AUTO-TERMINUS/repo_testing/glacier_files/BoxKangerlussuaq/BoxKangerlussuaq_WGS.shp /Users/jukesliu/Documents/AUTO-TERMINUS/repo_testing/glacier_files/BoxKangerlussuaq/BoxKangerlussuaq.shp
ogr2ogr -f 'ESRI Shapefile' -t_srs EPSG:4326 -s_srs EPSG:3413 /Users/jukesliu/Documents/AUTO-TERMINUS/repo_testing/glacier_files/BoxBernstorff1/BoxBernstorff1_WGS.shp /Users/jukesliu/Documents/AUTO-TERMINUS/repo_testing/glacier_files/BoxBernstorff1/BoxBernstorff1.shp
ogr2ogr -f 'ESRI Shapefile' -t_srs EPSG:4326 -s_srs EPSG:3413 /Users/jukesliu/Documents/AUTO-TERMINUS/repo_testing/glacier_files/BoxBernstorff2/BoxBernstorff2_WGS.shp /Users/jukesliu/Documents/AUTO-TERMINUS/repo_testing/glacier_files/BoxBernstorff2/BoxBernstorff2.shp
ogr2ogr -f 'ESRI Shapefile' -t_srs EPSG:4326 -s_srs EPSG:3413 /Users/jukesliu/Documents/AUTO-TERMINUS/repo_testing/glacier_files/BoxBernstorff3/BoxBernstorff3_WGS.shp /Users/jukesliu/Docum

In [14]:
# Grab the WGS84 coordinates of the boxes
box_points = {} # dictionary of points
for BoxID in BoxIDs:
    boxpath = basepath+"Box"+BoxID+"/Box"+BoxID
    termbox = fiona.open(boxpath+'_WGS.shp') # open reprojected terminus boxes
    box = termbox.next(); box_geom= box.get('geometry'); box_coords = box_geom.get('coordinates')[0] # grab coords
    points = [] # to hold the box vertices
    
    for coord_pair in box_coords:
        lat = coord_pair[0]; lon = coord_pair[1]        
        point = shapely.geometry.Point(lat, lon) # create shapely point 
        points.append(point) # append to points list
        
    box_points.update({BoxID: points}) # update dictionary
    print("Box"+BoxID+" done.") # keep track of progress

BoxKangerlussuaq done.
BoxBernstorff1 done.
BoxBernstorff2 done.
BoxBernstorff3 done.


<ipython-input-14-e8cf383faabf>:6: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  box = termbox.next(); box_geom= box.get('geometry'); box_coords = box_geom.get('coordinates')[0] # grab coords


In [19]:
# open the kml file with the Landsat path, row footprints:
WRS = fiona.open(basepath+'WRS-2_bound_world_0.kml', driver='KML')

In [20]:
#create lists to hold the paths and rows and BoxIDs
paths = []; rows = []; boxes = []

#loop through all features (path, row footprints)
for feature in WRS:
    # create shapely polygons from the Landsat footprints
    coordinates = feature['geometry']['coordinates'][0]
    coords = [xy[0:2] for xy in coordinates]
    pathrow_poly = Polygon(coords)
    
    # grab the path and row name from the WRS kml file:
    pathrowname = feature['properties']['Name']  
    path = pathrowname.split('_')[0]; row = pathrowname.split('_')[1]
#     print(path, row)
    
    #for each feature, loop through each of the box_points
    for BoxID in box_points:  
        box_points_in = 0 # counter for number of box_points in the pathrow_geom:
        points = box_points.get(BoxID) # grab the points corresponding to the ID
        for i in range(0, len(points)):
            point = points[i]
            # if the pathrow shape contains the point
            if point.within(pathrow_poly):
                box_points_in = box_points_in+1 # append the counter
        if box_points_in == 5: # if all box vertices are inside the footprint, save the path, row, BoxID
            paths.append('%03d' % int(path))
            rows.append('%03d' % int(row))
            boxes.append(BoxID)

# Store in dataframe
boxes_pr_df = pd.DataFrame(list(zip(boxes, paths, rows)), columns=['BoxID','Path', 'Row'])
boxes_pr_df = boxes_pr_df.sort_values(by='BoxID')
boxes_pr_df # display

,BoxID,Path,Row
0,Bernstorff1,232,016
1,Bernstorff1,001,015
4,Bernstorff1,233,015
7,Bernstorff1,232,015
2,Bernstorff2,001,015
5,Bernstorff2,233,015
8,Bernstorff2,232,015
3,Bernstorff3,001,015
6,Bernstorff3,233,015
9,Bernstorff3,232,015


In [8]:
######################################################################################
PR_FILENAME = 'LS_pathrows_Tess.csv' # change the filename as desired
######################################################################################
boxes_pr_df.to_csv(path_or_buf = basepath+PR_FILENAME, sep=',') # write to csv

# 3) Download Landsat metadata files from AWS for overlapping scenes
The aws syntax for grabbing an individual Landsat 8 metadata file is as follows:

     aws --no-sign-request s3 cp s3://landsat-pds/c1/L8/path/row/LC08_XXXX_pathrow_yyyyMMdd_01_T1/LC08_XXXX_pathrow_yyyyMMdd_01_T1_MTL.txt /path_to/output/

We can use the paths and rows in the dataframe to access the full Landsat 8 scene list and the corresponding metdata files. Read https://docs.opendata.aws/landsat-pds/readme.html to learn more.

Download the metadatafiles into Path_Row folders using:

    aws --no-sign-request s3 cp s3://landsat-pds/c1/L8/031/005/ Output/path/LS8aws/Path031_Row005/ --recursive --exclude "*" --include "*MTL.txt" 

In [9]:
# Read in csv file from Step 2
boxes_pr_df = pd.read_csv(basepath+PR_FILENAME, dtype=str)
boxes_pr_df = boxes_pr_df.set_index('BoxID'); boxes_pr_df

,Unnamed: 0,Path,Row,Zone
BoxID,,,,
HM,1,029,006,19
HM,2,028,006,19
HM,16,030,005,20
HM,13,033,005,19
HM,14,032,005,19
HM,15,031,005,19
IS,17,027,005,20
IS,3,027,006,20
IS,4,026,006,20


In [31]:
# Loop through the dataframe containing overlapping path, row info:
for index, row in boxes_pr_df.iterrows():
    p = row['Path']; r = row['Row']
    folder_name = 'Path'+p+'_Row'+r+'_c1' # folder name

    bp_in = 's3://landsat-pds/c1/L8/' # path to access the Landsat 8 images on AWS
    totalp_in = bp_in+p+'/'+r+'/' # adds path and row folders to the path
    bp_out = downloadpath+folder_name+'/' # output path for the downloaded files
    
    # create Path_Row folders if they don't exist already
    if os.path.exists(bp_out):
        print(folder_name, " EXISTS ALREADY. SKIP.")
    else:
        os.mkdir(bp_out)
        print(folder_name+" directory made")
        
        # construct command to download all metadata files
        command = 'aws --no-sign-request s3 cp '+totalp_in+' '+bp_out+' --recursive --exclude "*" --include "*MTL.txt"'
        subprocess.call(command, shell=True) # run on terminal

Path232_Row016_c1 directory made
Path001_Row015_c1 directory made
Path233_Row015_c1 directory made
Path232_Row015_c1 directory made
Path001_Row015_c1  EXISTS ALREADY. SKIP.
Path233_Row015_c1  EXISTS ALREADY. SKIP.
Path232_Row015_c1  EXISTS ALREADY. SKIP.
Path001_Row015_c1  EXISTS ALREADY. SKIP.
Path233_Row015_c1  EXISTS ALREADY. SKIP.
Path232_Row015_c1  EXISTS ALREADY. SKIP.
Path232_Row012_c1 directory made
Path231_Row012_c1 directory made
Path230_Row012_c1 directory made
Path229_Row012_c1 directory made
Path232_Row011_c1 directory made


# 4) Calculate cloud % over terminus box using Landsat quality band

If the terminus box shapefiles were not originally in UTM projection, will need to reproject them into UTM to match the Landsat 8 projection. The code automatically finds the UTM zones from the metadata files and fills in the following syntax to reproject:
    
    ogr2ogr -f "ESRI Shapefile" -t_srs EPSG:326zone -s_srs EPSG:3413 output.shp input.shp
    
If they are already in UTM projection, skip this step and rename the files to end with "\_UTM\_##.shp" where ## corresponds to the zone number (e.g., "\_UTM\_07.shp", "\_UTM\_21.shp").

In [45]:
zones = {} # initialize dictionary to hold UTM zone for each Landsat scene path row
zone_list = [] # list of zones

# Loop through all scenes:
for index, row in boxes_pr_df.iterrows():
    p = row['Path']; r = row['Row']
    BoxID = str(index)
    folder_name = 'Path'+p+'_Row'+r+'_c1'
    pr_folderpath = downloadpath+folder_name+'/' # path to the downloaded metadata files
    pathtoshp = basepath+"Box"+BoxID+"/Box"+BoxID # path to the terminus box shapefiles (all projections)
    
    if len(os.listdir(pr_folderpath)) > 0: # if there are files in the folder
        # grab UTM Zone from the first metadata file
        mtl_scene = os.listdir(pr_folderpath)[0]
        mtl = open(pr_folderpath+mtl_scene+'/'+mtl_scene+'_MTL.txt', 'r')
        for line in mtl:  #loop through lines in metadata to find the UTM ZONE
            variable = line.split("=")[0]
            if ("UTM_ZONE" in variable): 
                #save it:
                zone = '%02d' % int(line.split("=")[1][1:-1])
                zones.update({folder_name: zone}); zone_list.append(zone)
                
        # reproject shapefile(s) into UTM
        zone = zones[folder_name]
        rp_shp = 'ogr2ogr -f "ESRI Shapefile" '+pathtoshp+'_UTM_'+zone+'.shp '+pathtoshp+'_WGS.shp -t_srs EPSG:326'+zone+' -s_srs EPSG:4326'
        print(rp_shp)
        subprocess.call(rp_shp, shell=True)
        
    else: # if no files in folder, zone = nan, must fill in manually
        zone_list.append(np.nan)
        
boxes_pr_df['Zone'] = zone_list # add to the path row dataframe
print("Done.")

ogr2ogr -f "ESRI Shapefile" /Users/jukesliu/Documents/AUTO-TERMINUS/repo_testing/glacier_files/BoxBernstorff1/BoxBernstorff1_UTM_24.shp /Users/jukesliu/Documents/AUTO-TERMINUS/repo_testing/glacier_files/BoxBernstorff1/BoxBernstorff1_WGS.shp -t_srs EPSG:32624 -s_srs EPSG:4326
ogr2ogr -f "ESRI Shapefile" /Users/jukesliu/Documents/AUTO-TERMINUS/repo_testing/glacier_files/BoxBernstorff1/BoxBernstorff1_UTM_23.shp /Users/jukesliu/Documents/AUTO-TERMINUS/repo_testing/glacier_files/BoxBernstorff1/BoxBernstorff1_WGS.shp -t_srs EPSG:32623 -s_srs EPSG:4326
ogr2ogr -f "ESRI Shapefile" /Users/jukesliu/Documents/AUTO-TERMINUS/repo_testing/glacier_files/BoxBernstorff1/BoxBernstorff1_UTM_24.shp /Users/jukesliu/Documents/AUTO-TERMINUS/repo_testing/glacier_files/BoxBernstorff1/BoxBernstorff1_WGS.shp -t_srs EPSG:32624 -s_srs EPSG:4326
ogr2ogr -f "ESRI Shapefile" /Users/jukesliu/Documents/AUTO-TERMINUS/repo_testing/glacier_files/BoxBernstorff1/BoxBernstorff1_UTM_24.shp /Users/jukesliu/Documents/AUTO-TERMI

In [46]:
# resave path row csv file with UTM zone information, see above for variable PR_FILENAME
boxes_pr_df.to_csv(path_or_buf = basepath+PR_FILENAME, sep=',')

Use gdalwarp commands and the __vsicurl__ link to download subset of the quality band we will use to determine cloud cover:

    gdalwarp -cutline path_to_shp.shp -crop_to_cutline /vsicurl/https://landsat-pds.s3.amazonaws.com/c1/L8/031/005/imagename/imagename_BQA.TIF path_to_subsetBQA.TIF
    
These subset BQA files will go into the generated BoxID folders.

In [64]:
# Loop through all scenes:
for index, row in boxes_pr_df.iterrows():
    p = row['Path']; r = row['Row']
    zone = row['Zone'] # new zone data
    BoxID = str(index)
    folder_name = 'Path'+p+'_Row'+r+'_c1'
    pr_folderpath = downloadpath+folder_name+'/' # path to the downloaded metadata files
    pathtoshp = basepath+"Box"+BoxID+"/Box"+BoxID # path to the terminus box shapefiles (all projections)

    scenes = os.listdir(pr_folderpath) # grab the names of the LS8 scenes

    for scene in scenes:
        if len(scenes) > 0 and scene.startswith("LC08") and 'L1TP' in scene: # L1TP scenes
            pathtoshp_rp = pathtoshp+'_UTM_'+zone # path to the UTM box shapefile
            
            #set path to the quality band (BQA.TIF) in the cloud using the scene name
            pathtoBQA = '/vsicurl/https://landsat-pds.s3.amazonaws.com/c1/L8/'+p+'/'+r+'/'+scene+"/"+scene+"_BQA.TIF"
            subsetout = pr_folderpath+scene+"/"+scene+'_BQA_Box'+BoxID+'.TIF' # output file naming

            # construct download command
            BQA_dwnld_cmd='gdalwarp -cutline '+pathtoshp_rp+'.shp -crop_to_cutline '+pathtoBQA+' '+subsetout
#             print(BQA_dwnld_cmd) # check command syntax before downloading
            
            # Uncomment the following to run:
            subprocess.call(BQA_dwnld_cmd, shell=True)
            print(scene+'_BQA_Box'+BoxID+'.TIF subset downloaded')

LC08_L1TP_232016_20191015_20191015_01_RT_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_232016_20190828_20190828_01_RT_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_232016_20190727_20190801_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_232016_20180302_20180302_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1GT_232016_20190422_20190422_01_RT_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_232016_20210310_20210317_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_232016_20200408_20200408_01_RT_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1GT_232016_20190524_20190604_01_T2_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_232016_20210614_20210614_01_RT_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_232016_20170603_20170615_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_232016_20141102_20180528_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_232016_20180302_20180319_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_232016

LC08_L1TP_232016_20160803_20170322_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_232016_20190625_20190705_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_232016_20150630_20170407_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_232016_20191015_20191029_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_232016_20180809_20180815_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1GT_232016_20180606_20180606_01_RT_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_232016_20160429_20170326_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_232016_20170907_20170907_01_RT_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1GT_232016_20191202_20191202_01_RT_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_232016_20140915_20170419_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1GT_232016_20180521_20180521_01_RT_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1GT_232016_20170619_20170619_01_RT_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_232016

LC08_L1TP_001015_20180127_20180127_01_RT_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_001015_20140711_20170421_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_001015_20160902_20170321_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_001015_20210511_20210511_01_RT_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_001015_20131028_20170429_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_001015_20160614_20170324_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_001015_20130825_20170502_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_001015_20201015_20201104_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1GT_001015_20170921_20170921_01_RT_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_001015_20210409_20210416_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_001015_20180924_20180924_01_RT_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_001015_20170617_20170629_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1GT_001015

LC08_L1TP_233015_20170728_20170729_01_RT_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_233015_20130903_20170502_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_233015_20131005_20170429_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_233015_20170829_20170914_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_233015_20180629_20180629_01_RT_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1GT_233015_20210605_20210605_01_RT_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_233015_20150808_20180528_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1GT_233015_20210213_20210213_01_RT_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_233015_20200227_20200227_01_RT_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_233015_20170930_20171013_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_233015_20210621_20210629_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_233015_20200602_20200602_01_RT_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_233015

LC08_L1GT_233015_20190429_20190429_01_RT_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_233015_20150723_20170406_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_233015_20171117_20171117_01_RT_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1GT_233015_20170525_20170525_01_RT_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_233015_20180715_20180715_01_RT_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_233015_20140906_20170419_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_233015_20190208_20190208_01_RT_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_233015_20190803_20190819_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1GT_233015_20180309_20180309_01_RT_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_233015_20131122_20170428_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_233015_20200821_20200905_01_T1_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_233015_20210520_20210520_01_RT_BQA_BoxBernstorff1.TIF subset downloaded
LC08_L1TP_233015

...

LC08_L1TP_231012_20150506_20170409_01_T1_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_231012_20160828_20170321_01_T1_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_231012_20160321_20170327_01_T1_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_231012_20160913_20180528_01_T1_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_231012_20171002_20171002_01_RT_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_231012_20200316_20200326_01_T1_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_230012_20191102_20191102_01_RT_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_230012_20210515_20210515_01_RT_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_230012_20130330_20170505_01_T1_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_230012_20130325_20170505_01_T1_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_230012_20140816_20170420_01_T1_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_230012_20150429_20170409_01_T1_BQA_BoxKangerlussuaq.TIF subset dow

LC08_L1TP_230012_20130914_20170502_01_T1_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_230012_20150702_20170407_01_T1_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_230012_20181030_20181030_01_RT_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_230012_20200222_20200222_01_RT_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_230012_20181014_20181030_01_T1_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_230012_20180912_20180927_01_T1_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_230012_20150616_20170407_01_T1_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_230012_20170707_20170707_01_RT_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_230012_20160330_20170327_01_T1_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_230012_20190203_20190203_01_RT_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_230012_20200512_20200512_01_RT_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_230012_20180827_20180911_01_T1_BQA_BoxKangerlussuaq.TIF subset dow

LC08_L1TP_229012_20180209_20180209_01_RT_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_229012_20210422_20210501_01_T1_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_229012_20160814_20180528_01_T1_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_229012_20150913_20170404_01_T1_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_229012_20160729_20170322_01_T1_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_229012_20190620_20190620_01_RT_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_229012_20191026_20191030_01_T1_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_229012_20140622_20170421_01_T1_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_229012_20180719_20180731_01_T1_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_229012_20200606_20200606_01_RT_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_229012_20190722_20190722_01_RT_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1GT_229012_20190908_20190908_01_RT_BQA_BoxKangerlussuaq.TIF subset dow

LC08_L1TP_229012_20200708_20200721_01_T1_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_229012_20170529_20180528_01_T1_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_232011_20200510_20200526_01_T1_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_232011_20190305_20190305_01_RT_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_232011_20200526_20200608_01_T1_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_232011_20210630_20210701_01_RT_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_232011_20200814_20200822_01_T1_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_232011_20150614_20170407_01_T1_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_232011_20170705_20170705_01_RT_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_232011_20200323_20200323_01_RT_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_232011_20200510_20200510_01_RT_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_232011_20190609_20190609_01_RT_BQA_BoxKangerlussuaq.TIF subset dow

LC08_L1GT_232011_20180910_20180910_01_RT_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_232011_20190812_20190812_01_RT_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_232011_20160819_20170322_01_T1_BQA_BoxKangerlussuaq.TIF subset downloaded
LC08_L1TP_232011_20190508_20190521_01_T1_BQA_BoxKangerlussuaq.TIF subset downloaded


# 5) Create buffer zone around terminus boxes and rasterize terminus boxes

In [3]:
buffers = []
mindimensions = []

# Calculate a buffer distance around the terminus box:
for BoxID in BoxIDs:
    for file in os.listdir(basepath+'Box'+BoxID+'/'):
        if 'UTM' in file and '.shp' in file and "Box" in file: # identify UTM projected box
            boxpath = basepath+"Box"+BoxID+"/"+file  
            termbox = fiona.open(boxpath)
            
    # grab the box coordinates:
    box = termbox.next(); box_geom= box.get('geometry'); box_coords = box_geom.get('coordinates')[0]
    points = []
    for coord_pair in box_coords:
        lat = coord_pair[0]; lon = coord_pair[1]; points.append([lat, lon])
            
    # Calculate distance between coord 1 and 2 and between 2 and 3
    coord1 = points[0]; coord2 = points[1]; coord3 = points[2]   
    dist1 = distance(coord1[0], coord1[1], coord2[0], coord2[1]);
    dist2 = distance(coord2[0], coord2[1], coord3[0], coord3[1]) 
    buff_dist = int(np.max([dist1, dist2])) # pick the longer one as the buffer distance
    mindim = int(np.min([dist1, dist2]))/15.0 # calculate the minimum dimension in pixels
    
    # record:
    buffers.append(buff_dist)
    mindimensions.append(int(mindim))

# store as dataframe:
buff_df = pd.DataFrame(list(zip(BoxIDs, buffers, mindimensions)), columns=['BoxID', 'Buff_dist_m', 'min_dim_px'])
buff_df

/home/jukes/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  if sys.path[0] == '':


,BoxID,Buff_dist_m,min_dim_px
0,HM,18769,175
1,Petermann,26469,630
2,Sverdrup,19466,309


In [4]:
######################################################################################
BOX_FILENAME = 'Buffdist_Tess.csv'
buff_df.to_csv(basepath+BOX_FILENAME) # write to csv, change name as desired
######################################################################################

The next section creates a buffer zone shapefile using GDAL command **ogr2ogr** with the following syntax:

    ogr2ogr Buffer###.shp path_to_terminusbox###.shp  -dialect sqlite -sql "SELECT ST_Buffer(geometry, buffer_distance) AS geometry,*FROM 'Box###'" -f "ESRI Shapefile"

In [5]:
# loop through the buffer distance dataframe:
for index, row in buff_df.iterrows():
    BoxID = row['BoxID']
    buff_dist = str(row['Buff_dist_m'])
    
    terminusbox_path = basepath+"Box"+BoxID+"/Box"+BoxID+".shp" # path to box shapefile
    outputbuffer_path = basepath+"Box"+BoxID+"/Buffer"+BoxID+".shp" # path and name of new buffer file
    
    # Set buffer command
    buffer_cmd = 'ogr2ogr '+outputbuffer_path+" "+terminusbox_path+' -dialect sqlite -sql "SELECT ST_Buffer(geometry, '+buff_dist+") AS geometry,*FROM 'Box"+BoxID+"'"+'" -f "ESRI Shapefile"'
    print(buffer_cmd)
    
    subprocess.call(buffer_cmd, shell=True) # run on terminal

ogr2ogr /home/jukes/Documents/Sample_glaciers/BoxHM/BufferHM.shp /home/jukes/Documents/Sample_glaciers/BoxHM/BoxHM.shp -dialect sqlite -sql "SELECT ST_Buffer(geometry, 18769) AS geometry,*FROM 'BoxHM'" -f "ESRI Shapefile"
ogr2ogr /home/jukes/Documents/Sample_glaciers/BoxPetermann/BufferPetermann.shp /home/jukes/Documents/Sample_glaciers/BoxPetermann/BoxPetermann.shp -dialect sqlite -sql "SELECT ST_Buffer(geometry, 26469) AS geometry,*FROM 'BoxPetermann'" -f "ESRI Shapefile"
ogr2ogr /home/jukes/Documents/Sample_glaciers/BoxSverdrup/BufferSverdrup.shp /home/jukes/Documents/Sample_glaciers/BoxSverdrup/BoxSverdrup.shp -dialect sqlite -sql "SELECT ST_Buffer(geometry, 19466) AS geometry,*FROM 'BoxSverdrup'" -f "ESRI Shapefile"


The terminus box shapefiles are then rasterized (to be used as a mask during the WTMM filering) using the GDAL **gdal_rasterize** command, subset to the buffer zone using the GDAL **gdalwarp** command, and reprojected:

1) Rasterize

        gdal_rasterize -burn 1.0 -tr x_resolution y_resolution -a_nodata 0.0 path_to_terminusbox.shp path_to_terminusbox_raster.TIF
    
2) Subset

        gdalwarp -cutline path_to_Buffer###.shp -crop_to_cutline path_to_terminusbox_raster.TIF path_to_subset_raster_cut.TIF
    
3) Reproject to UTM

In [10]:
######################################################################################
source_srs = 'ESPG:3413' # original projection of the shapfiles (Greenland polar stereo = 3413)
######################################################################################

for index, row in buff_df.iterrows():
    BoxID = row['BoxID']
    terminusbox_path = basepath+"Box"+BoxID+"/Box"+BoxID # path to box
    buffer_path = basepath+"Box"+BoxID+"/Buffer"+BoxID # path to buffer
    terminusraster_path = basepath+"Box"+BoxID+"/Box"+BoxID+".TIF" # path to rasterized box
    cutraster_path = basepath+"Box"+BoxID+"/Box"+BoxID+"_raster_cut.TIF" # name for cropped file
    
    zone = boxes_pr_df.loc[BoxID, 'Zone'] # grab zone matching BoxID from other dataframe
    
    # Set commands
    rasterize_cmd = 'gdal_rasterize -burn 1.0 -tr 15.0 15.0 -a_nodata 0.0 '+terminusbox_path+'.shp '+terminusraster_path+'.shp'
    subsetbuffer_cmd = 'gdalwarp -cutline '+buffer_path+' -crop_to_cutline '+terminusraster_path+'.shp '+cutraster_path+'.shp'
    rp_shp = 'ogr2ogr -f "ESRI Shapefile" -t_srs EPSG:326'+zone+' -s_srs EPSG:'+source_srs+' '+buffer_path+"_UTM_"+zone+".shp "+buffer_path+'.shp'
    
    subprocess.call(rasterize_cmd, shell=True) # rasterize with command terminal
    subprocess.call(subsetbuffer_cmd, shell=True) # subset to buffer with command terminal
    subprocess.call(rp_shp, shell=True) # reproject
    
    print("Box"+BoxID) # check progress

BoxHM


TypeError: expected str, bytes or os.PathLike object, not float

# 6) Download non-cloudy Landsat images from AWS

To remove cloudy images, we will find the number of pixels in our terminux box that correspond to a cumulative pixel value of > 4096 in the BQA band. If the fraction of cloudy pixels with values is above the threshold, we won't download the image. 

Additionally, we remove images that are primarily black (fill value of 0 or 1 in BQA band). This ensures that the scenes that cut off halfway across the glacier are not included in further analysis. The fill percent threshold may need to be adjusted.

In [21]:
######################################################################################
# These are the recommended values. Adjust thresholds here:
BQA_thresh = 4096.0 # BQA pixel value threshold to be considered cloud
cpercent_thresh = 20.0 # maximum cloud cover % in terminus box
fpercent_thresh = 60.0 # maximum fill % in terminux box

# Set your desired Landsat bands to download:
bands = [8] # panchromatic
# bands = [2, 3, 4] # multispectral

######################################################################################

In [ ]:
# Download images that pass these thresholds:
for index, row in boxes_pr_df.iterrows():
    p = row['Path']; zone = row['Zone']; r = row['Row']; BoxID = index; 
    folder_name = 'Path'+p+'_Row'+r+'_c1'
    pr_folderpath = downloadpath+folder_name+'/'
    
    bp_out = downloadpath+'Box'+BoxID+'/' # folder name for downloaded images
    if os.path.exists(bp_out): # create folder if it does not exist
        print("Box"+BoxID, " exists already. Skip creation of directory.")
    else:
        os.mkdir(bp_out)
        print("Box"+BoxID+" directory made.")
    
    # path to the shapefile covering the region that will be downloaded
    pathtobuffer = basepath+'Box'+BoxID+'/Buffer'+BoxID+'_UTM_'+zone+'.shp'  # buffer around box - recommended
#     pathtobox = basepath+'Box'+BoxID+'/Box'+BoxID+'_UTM_'+zone+'.shp' # just the box
    
    for scene in os.listdir(pr_folderpath):
        if len(os.listdir(pr_folderpath)) > 0 and scene.startswith("LC08"):
            BQApath = pr_folderpath+scene+"/"+scene+'_BQA_Box'+BoxID+'.TIF' # path to BQA file
            subsetBQA = mpimg.imread(BQApath) # read in as numpy array
            
            totalpixels = subsetBQA.shape[0]*subsetBQA.shape[1] # total number of pixels
            cloudBQA = subsetBQA[subsetBQA > BQA_thresh] # cloudy pixels (value > BQA_thresh)
            fillBQA = subsetBQA[subsetBQA < 2.0] # fill pixels (value = 0 or 1)
            cloudpixels = len(cloudBQA); fillpixels = len(fillBQA) # count the cloudy and fill pixels
            cloudpercent = int(float(cloudpixels)/float(totalpixels)*100) # calculate percent cloudy
            fillpercent = int(float(fillpixels)/float(totalpixels)*100) # calculate percent fill
            print(scene, 'Cloud % ', cloudpercent, 'Fill %', fillpercent) # check values
            
            # evaluate thresholds
            if cloudpercent <= cpercent_thresh and fillpercent <= fpercent_thresh:
                # download the bands for that scene into your scene folders:
                for band in bands:
                    band = str(band) # string format
                    
                    # input path to your bands in AWS:
                    pathin = '/vsicurl/https://landsat-pds.s3.amazonaws.com/c1/L8/'+p+"/"+r+"/"+scene+"/"+scene+"_B"+band+".TIF"
                    # output path to the overall BoxID folder
                    outfilename = scene+"_B"+band+'_Buffer'+BoxID+'.TIF'
                    pathout = downloadpath+'Box'+BoxID+'/'+outfilename
                    
                    # construct download command
                    download_cmd = 'gdalwarp -cutline '+pathtobuffer+' -crop_to_cutline '+pathin+' '+pathout
#                     download_cmd = 'gdalwarp -cutline '+pathtobox+' -crop_to_cutline '+pathin+' '+pathout
                    print(download_cmd) # check
                    
#                     # Once checked, uncomment the following to commence download:                   
#                     subprocess.call(download_cmd, shell=True)
#                     print(outfilename+" downloaded")

### Reproject the downloaded files from UTM into your desired projection

In [ ]:
# Reproject newly downloaded images:
for BoxID in list(set(boxes_pr_df.index)):
    bp_out = downloadpath+'Box'+BoxID+'/' # path to downloaded files
    
    # create output reprojected folder if does not exist
    if os.path.exists(bp_out+'reprojected/'):
        print("Box"+BoxID, "Reprojected EXISTS ALREADY.")
    else:
        os.mkdir(bp_out+'reprojected/')
        print("Box"+BoxID+" Reprojected directory made")
                      
    downloadedimages = os.listdir(bp_out) # all downloaded images
    for image in downloadedimages:
        if image.startswith("LC08"):
            imagename = image[:-4] # remove suffix
            
            ######################################################################################
            # Gdalwarp command must be re-written for other projections.
            # The command and image name suffix is for Greenland Polar Stereo projection
            suffix = '_PS'
            rp_PS = "gdalwarp -t_srs EPSG:3413 "+bp_out+image+" "+bp_out+'reprojected/'+imagename+suffix+".TIF"
            subprocess.call(rp_PS, shell=True)   
            
            ######################################################################################
            
    print("Box"+BoxID+" reprojected.")

### Automatically grab the image acquisition dates from the metadata files

In [ ]:
datetimes = [] # list of scene datetimes
scenes_dated = [] # list of scenes

# Loop through the dataframe with your path row combinations:
for index, row in boxes_pr_df.iterrows():
    p = row['Path']; r = row['Row']; BoxID = index; 
    folder_name = 'Path'+p+'_Row'+r+'_c1'; print(folder_name)
    
    # Output folder paths"
    folderpath = downloadpath+folder_name+'/'
    bp_out = downloadpath+'Box'+BoxID+'/reprojected/'
      
    scenecount = 0 # keep track of the number of scenes:
    
    downloaded_scenes = os.listdir(bp_out)
    for scene in downloaded_scenes:
        if scene.endswith('.TIF') and scene.startswith("LC08"):
            scenename = scene[:-20] # MAY NEED TO ADJUST DEPENDING ON SUFFIX - CHANGE THIS TO start and end
            if scenename in os.listdir(folderpath):
                scenefiles = os.listdir(folderpath+scenename+'/')
                for file in scenefiles:
                    if ("MTL.txt" in file): # open metadata file
                        mdata = open(folderpath+scenename+"/"+scenename+"_MTL.txt", "r")
                        for line in mdata:
                            variable = line.split("=")[0]
                            if ("DATE_ACQUIRED" in variable):
                                date = line.split("=")[1][1:-1] # find acquisition date

                        dates = datetime.datetime.strptime(date, '%Y-%m-%d') # save as datetime object
                        datetimes.append(dates); scenes_dated.append(scenename) # store in lists
                scenecount = scenecount+1

# Store in a dataframe
datetime_df = pd.DataFrame(list(zip(scenes_dated, datetimes)), columns=['Scene', 'datetime'])
datetime_df = datetime_df.sort_values(by='datetime', ascending=True); datetime_df = datetime_df.drop_duplicates()
datetime_df

In [ ]:
######################################################################################
DATES_FILENAME = 'imgdates_nonSE.csv' # change image date csv filename as desired
######################################################################################
datetime_df.to_csv(path_or_buf = basepath+DATES_FILENAME, sep=',') # write to csv

### Second pass at cloud filtering using range in pixel values

Removes cloudy images that slipped through BQA filtering. Skip if unnecessary.

In [24]:
# # convert all files in reprojected folder to png from TIF
# for BoxID in BoxIDs:
#     print(BoxID)
#     command = 'cd '+downloadpath+'Box'+BoxID+'/reprojected/; '+'mogrify -format png *_PS.TIF'
#     subprocess.call(command, shell=True)

In [26]:
# ######################################################################################
# suffix = '_PS' # reprojection suffix
# ######################################################################################

# for BoxID in BoxIDs:
#     imagepath = downloadpath+'Box'+BoxID+'/reprojected/'
#     for img in os.listdir(imagepath):
#         if img.endswith('Buffer'+BoxID+suffix+'.png'):
#             image = cv2.imread(imagepath+img,-1) # read in image
#             imageplt = mpimg.imread(imagepath+img)
#             image_nofill = imageplt[imageplt > 0] # don't consider the fill points
#             img_std = np.std(image_nofill) # st. dev in values
#             if len(image_nofill.shape) > 1:
#                 img_range = np.max(image_nofill) - np.min(image_nofill)
#                 img_med = np.median(image_nofill)

#                 if img_std < 0.04 and img_med > 0.15: # adjust these threholds
#                     os.remove(imagepath+img) # remove png mimage
#                     os.remove(imagepath+img[:-4]+'.TIF') # remove pgm image     
#                     # show the image
#     #                 imgplt_trim = plt.imshow(cv2.cvtColor(imageplt, cv2.COLOR_BGR2RGB))
#     #                 plt.show()

### Grab fraction of total images available that were excluded due to clouds and fill

If you are interested in knowing how many images were filtered out using the cloud and fill thresholds, run the following cells. Otherwise, skip.

In [ ]:
# read in path, row csv file if not already loaded
boxes_pr_df = pd.read_csv(basepath+PR_FILENAME, dtype=str)
boxes_pr_df = boxes_pr_df.set_index('BoxID'); BoxIDs = list(set(boxes_pr_df.index))
print(BoxIDs); boxes_pr_df.head()

In [ ]:
im_tots = []
downloaded = []
fractions = []

for BoxID in BoxIDs:
    pathrows_BoxID = boxes_pr_df[boxes_pr_df.index == BoxID].copy() # grab path rows for that BoxID
    
    im_tot = 0 # count number of total scenes available
    for idx, rw in pathrows_BoxID.iterrows():
        p = rw['Path']; r = rw['Row']
        ims_pr = len(os.listdir(downloadpath+'Path'+p+'_Row'+r+'_c1')) # grab number of scenes in that pathrow
        im_tot = im_tot + ims_pr
    
    counter = 0
    if im_tot == 0: # if no images
        download_frac = np.NaN
    else:
        # count the files that passed thresholds and got downloaded
        for file in os.listdir(downloadpath+'Box'+BoxID+'/reprojected/'):
            ######################################################################################
            if file.startswith('LC08') and file.endswith('.png') and 'B8' in file: # panchromatic band (B8)
            # adjust the if statement if you are using a different band
            ######################################################################################
                counter = counter + 1
            
        download_frac = int(counter/im_tot*100) # calculate fraction downloaded
    im_tots.append(im_tot); downloaded.append(counter); fractions.append(download_frac) # store values

# store in dataframe
downloaded_df = pd.DataFrame(list(zip(BoxIDs, im_tots, downloaded, fractions)), columns = ['BoxID', 'Total_ims', 'Downloaded', '%'])
downloaded_df 

In [ ]:
######################################################################################
DOWNLOADED_FILENAME = 'Images_downloaded_nonSE.csv' # change csv file name as desired
######################################################################################
downloaded_df.to_csv(basepath+DOWNLOADED_FILENAME, sep=',') # write to csv

# 7) Calculate weighted average glacier flow direction using velocity data

The following code processes ice velocity (vx, vy) rasters to determine each glacier of interest's weighted average flow direction. These files should be placed in the base directory (basepath). The rasters are subset using the terminus box shapefile or the Randolph Glacier Inventory outlines using a GDAL command (**gdalwarp**) with the following syntax:

    gdalwarp -cutline path_to_terminusbox.shp -crop_to_cutline path_to_input_velocity.TIF path_to_output_velocity_at_term###.TIF

In [11]:
######################################################################################
# Change to your velocity input file names
vx_name = 'greenland_vel_mosaic250_vx_v1.tif' # MEaSUREs product
vy_name = 'greenland_vel_mosaic250_vy_v1.tif' # MEaSUREs product

######################################################################################

for BoxID in BoxIDs:
    ######################################################################################
    # It's okay if you don't have the Randolph Glacier Inventory outlines, but if you do,
    # adjust the path to them here
    terminus_path = basepath+"Box"+BoxID+"/RGI_Box"+BoxID+".shp"  # path to RGI shapefiles
    
    ######################################################################################
    if os.path.exists(terminus_path) == False: # if the RGI shapefile does not exist
        terminus_path = basepath+"Box"+BoxID+"/Box"+BoxID+".shp"  # set the path to the terminux box shapefiles      
        # output paths for the cropped velocity data:
        vx_out = basepath+"Box"+BoxID+"/Box"+BoxID+'_'+vx_name
        vy_out = basepath+"Box"+BoxID+"/Box"+BoxID+'_'+vy_name
    else:
        vx_out = basepath+"Box"+BoxID+"/RGI_Box"+BoxID+'_'+vx_name
        vy_out = basepath+"Box"+BoxID+"/RGI_Box"+BoxID+'_'+vy_name
    
    # input paths:
    vx_in = basepath+vx_name
    vy_in = basepath+vy_name
    
    # subset x and y velocity files:
    v_subset1 = 'gdalwarp -cutline '+terminus_path+' -crop_to_cutline '+vx_in+" "+vx_out
    v_subset2 = 'gdalwarp -cutline '+terminus_path+' -crop_to_cutline '+vy_in+" "+vy_out
    subprocess.call(v_subset1, shell=True)
    subprocess.call(v_subset2, shell=True)
    
    print("Box"+BoxID+' done.')

BoxHM done.
BoxPetermann done.
BoxSverdrup done.


Next, these subset velocity rasters are opened using the **rasterio** package and read into arrays. They are filtered for anomalous values and the velocity magnitudes are converted into weights. Then the **numpy.average()** function is used to calculated the weighted average flow directions where the flow directions of the pixels where the highest velocities are found are weighted more. 

The resulting average flow direction will be representative of the glacier's main flow. These directions will be used to rotate the images of the glaciers so that their flow is due right.

__For slow-moving glaciers with uncertain velocities from feature tracking based velocity datasets, use manual determination of velocities. Here, we use the manual delineations of the Greenland peripheral glaciers in 2000 and 2015 to approximate the flow direction.__

In [28]:
##################################################################################################################
# ONLY APPLIES TO GREENLAND PERIPHERAL GLACIERS
badvelocities = ['301', '289', '283', '265', '241', '223', '285', '181', '097', '091', '067','083',
                 '221', '173', '113', '101', '089', '082', '100', '112', '118', '130', '160', '196', 
                 '208', '226', '256', '262', '280', '298', '322', '072', '074', '080', '082', '084',
                '102', '114', '134', '132', '144', '159', '188', '189', '198', '207', '212', '222',
                '224', '234', '242', '243', '249', '254', '258', '264', '267', '272', '273', '278', 
                '282', '284', '288', '297', '305', '306', '307', '315', '318', '321', '324', '327',
                '330', '331', '338', '341', '344', '354', '356', '357', '358', '359', '362', '363',
                 '364', '369', '370', '371', '372', '373', '374', '376', '377', '379', '380', '381', 
                 '382', '383', '384', '385', '386', '387', '388', '389', '390', '391', '392', '393',
                 '394', '395', '396', '397', '398', '399', '400', '401' ,'404', '405', '406', '407',
                 '408', '409', '410', '414', '415', '416', '417', '418', '419', '420', '421', '422',
                 '427', '430', '431', '434', '436', '438', '440']
print(badvelocities)
##################################################################################################################

['301', '289', '283', '265', '241', '223', '285', '181', '097', '091', '067', '083', '221', '173', '113', '101', '089', '082', '100', '112', '118', '130', '160', '196', '208', '226', '256', '262', '280', '298', '322', '072', '074', '080', '082', '084', '102', '114', '134', '132', '144', '159', '188', '189', '198', '207', '212', '222', '224', '234', '242', '243', '249', '254', '258', '264', '267', '272', '273', '278', '282', '284', '288', '297', '305', '306', '307', '315', '318', '321', '324', '327', '330', '331', '338', '341', '344', '354', '356', '357', '358', '359', '362', '363', '364', '369', '370', '371', '372', '373', '374', '376', '377', '379', '380', '381', '382', '383', '384', '385', '386', '387', '388', '389', '390', '391', '392', '393', '394', '395', '396', '397', '398', '399', '400', '401', '404', '405', '406', '407', '408', '409', '410', '414', '415', '416', '417', '418', '419', '420', '421', '422', '427', '430', '431', '434', '436', '438', '440']


In [29]:
boxes = []; avg_rot = []; max_mag = []; num_cells = []

for BoxID in BoxIDs:
    rot_angles = []; max_magnitudes = []
    
    # determine if RGI outline was used to subset velocities
    rgi_exists = 0
    for file in os.listdir(basepath+"Box"+BoxID):
        if file.startswith('RGI'):
            rgi_exists = 1
            
    if rgi_exists == 1: # if yes, open those files    
        vx = rasterio.open(basepath+"Box"+BoxID+"/RGI_Box"+BoxID+'_'+vx_name, "r") # RGI
        vy = rasterio.open(basepath+"Box"+BoxID+"/RGI_Box"+BoxID+'_'+vy_name, "r") # RGI
    else: # if not, they were subset using the boxes. Open those files
        vx = rasterio.open(basepath+"Box"+BoxID+"/Box"+BoxID+'_'+vx_name, "r") # box 
        vy = rasterio.open(basepath+"Box"+BoxID+"/Box"+BoxID+'_'+vy_name, "r") # box

    vx_array = vx.read(); vy_array = vy.read() # read as numpy array
    vx_masked = vx_array[vx_array != -2000000000.0] # remove no data values (-2000000000.0)
    vy_masked = vy_array[vy_array != -2000000000.0]

    direction = np.arctan2(vy_masked, vx_masked)*180/np.pi # calculate flow direction
    # transform so any negative angles are placed on 0 to 360 scale:
    if len(direction[direction < 0]) > 0:
        direction[direction < 0] = 360.0+direction[direction < 0]

    magnitude = np.sqrt((vx_masked*vx_masked) + (vy_masked*vy_masked)) # calculate speed (flow magnitude)
    
    ncells = len(direction) # number of pixels
    if ncells > 0:
        # Determine if there are a large number of direction pixels with values > 200.0
        # If so, it's probably pointing East
        dir_range = direction.max() - direction.min()
        if dir_range > 200.0 and len(direction[direction > 200]): # if large range and values above 200
            direction[direction > 180] = direction[direction > 180] - 360.0 # transform those values on a negative scale
            # calculate weights (0 - 1) from magnitudes
            mag_range = magnitude.max() - magnitude.min()
            stretch = 1/mag_range; weights = stretch*(magnitude - magnitude.min()) # weights for averaging
            avg_dir = np.average(direction, weights=weights) # calculate average flow direction
            if avg_dir < 0: # if negative:
                avg_dir = avg_dir + 360.0 # transform back to 0 to 360 scale
        else:
            mag_range = magnitude.max() - magnitude.min(); stretch = 1/mag_range
            weights = stretch*(magnitude - magnitude.min())
            avg_dir = np.average(direction, weights=weights)
                
        ###############################################################################################################
        # Adjust to convert flow magnitude to meters per day depending on units in your 
        # velocity dataset
        yr_day_conv = 0.00273973 # conversion to m/d from m/a
        max_magnitude = magnitude.max()*yr_day_conv 
        ###############################################################################################################
        
    else: # no velocity pixels remaining once cropped
        avg_dir = np.NaN ; max_magnitude = np.NaN # no velocities to calculate this with
    
    ##################################################################################################################
    # ONLY APPLIES TO GREENLAND PERIPHERAL GLACIERS, COMMENT OUT FOR OTHER APPLICATIONS
    path2000_2015 = '/media/jukes/jukes1/2000_2015/'
    if BoxID in badvelocities:
        # grab the 2000 and 2015 delineation centroids:
        shp2000 = fiona.open(path2000_2015+'GreenlandPeriph_term2000_'+BoxID+'.shp'); feat2000= shp2000.next()
        lineshp2000 = LineString(feat2000['geometry']['coordinates'])
        cent2000 = np.array(lineshp2000.centroid)

        shp2015 = fiona.open(path2000_2015+'GreenlandPeriph_term2015_'+BoxID+'.shp'); feat2015= shp2015.next()
        lineshp2015 = LineString(feat2015['geometry']['coordinates'])
        cent2015 = np.array(lineshp2015.centroid)

        # grab displacements and use to calculate flow direction in degrees
        y = cent2000[1] - cent2015[1]
        x = cent2000[0] - cent2015[0]
        avg_dir = np.arctan2(y,x)*180/np.pi
        if avg_dir < 0:
            avg_dir = 360.0+avg_dir
         
        # if max_magnitude cannot be calculated from the velocity raster (pixels == 0)
        if ncells == 0:
            # use displacements and time to approximate speed in m/d
            yrs = 15.0
            max_magnitude = np.sqrt((y*y)+(x*x))/yrs*yr_day_conv
                
        ncells = np.NaN
    ##################################################################################################################
    
    # Append values to lists:
    avg_rot.append(avg_dir); max_mag.append(max_magnitude); boxes.append(BoxID); num_cells.append(ncells)  

# store the flow direction (rotation angle), maximum magnitude
velocities_df = pd.DataFrame(list(zip(boxes,avg_rot, max_mag, num_cells)), columns=['BoxID','Flow_dir', 'Max_speed', 'Pixels'])
velocities_df = velocities_df.sort_values(by='BoxID')
velocities_df # display

,BoxID,Flow_dir,Max_speed,Pixels
0,HM,116.616096,1.141788,473
1,Petermann,95.897408,3.216419,3810


In [4]:
##################################################################################################################
VEL_FILENAME = 'Glacier_vel_Tess.csv' # change velocity file name
##################################################################################################################
velocities_df.to_csv(path_or_buf = basepath+VEL_FILENAME, sep=',') # write to csv

# 8) Rotate all images by flow direction

Read in the glacier velocity file as velocities_df if not already loaded:

In [5]:
velocities_df = pd.read_csv(basepath+VEL_FILENAME, sep=',', dtype=str, usecols=[1,2,3,4])
velocities_df = velocities_df.set_index('BoxID')

In [24]:
# make directory for rotated images in BoxID folders if it doesn't already exist
for index, row in velocities_df.iterrows():
    BoxID = index
    if os.path.exists(downloadpath+"Box"+BoxID+'/rotated_c1/'):
        print("Already exists.")
    else:
        os.mkdir(downloadpath+"Box"+BoxID+'/rotated_c1/')
        print("Folder made for Box"+BoxID)

Already exists.
Already exists.
Already exists.


In [32]:
# move rasterized terminus box into reprojected folder, since it will also need to be rotated:
for index, row in velocities_df.iterrows():
    BoxID = index
    boxfile = 'Box'+BoxID+'_raster_cut.TIF'
    boxrasterpath = basepath+'Box'+BoxID+'/'+boxfile
    newpath = downloadpath+'Box'+BoxID+'/reprojected/'+boxfile
    shutil.copyfile(boxrasterpath, newpath)

In [33]:
# convert all images to png for rotation:
for index, row in velocities_df.iterrows():
    BoxID = index
    command = 'cd '+downloadpath+'Box'+BoxID+'/reprojected/; '+'mogrify -format png *raster_cut.TIF'
    subprocess.call(command, shell=True)

In [35]:
# rotate
for index, row in velocities_df.iterrows():
    BoxID = index
    print(BoxID) # keep track of progress
    for file in os.listdir(downloadpath+"Box"+BoxID+'/reprojected/'):
        if file.endswith('.png'):
            img  = Image.open(downloadpath+"Box"+BoxID+'/reprojected/'+file)
            rotated = img.rotate(-float(row['Flow_dir']))
            rotated.save(downloadpath+"Box"+BoxID+'/rotated_c1/R_'+file)

HM
Petermann


# 9) Crop all images to the same size

All input images will need to be the same size for the automated terminus detection analysis. The function resize_pngs resizes all png files within a folder to the minimum image dimensions found. The function crops around the edges, centering the image.

In [6]:
for BoxID in BoxIDs:
    resizepath = downloadpath+"Box"+BoxID+'/rotated_c1/' # path to rotated images
    resize_pngs(resizepath) # crop

OSError: read past end of file

In [4]:
##################################################################################################################
# FOR XSMURF ANALYSIS ONlY
# convert all final files to pgm
for index, row in velocities_df.iterrows():
    BoxID = index
    command = 'cd '+downloadpath+'Box'+BoxID+'/rotated_c1/; '+'mogrify -depth 16 -format pgm *.png'
    subprocess.call(command, shell=True)
##################################################################################################################

In [22]:
# # rename the rasterized terminus box files if necessary
# for BoxID in BoxIDs:
#     files = os.listdir(downloadpath+'Box'+BoxID+'/rotated_c1/')
#     for file in files:
#         if file.startswith('R_Box'+BoxID+'_cut'):
#             rpath = downloadpath+'Box'+BoxID+'/rotated_c1/'
#             os.rename(rpath+file, rpath+'R_Box'+BoxID+'_raster_cut'+file[-4:])